In [ ]:
%matplotlib inline
import sys, time
import pyopencl as cl
import tables
import numpy as np
import scipy.misc as scp
import matplotlib.pyplot as plt

In [ ]:
def init_data(population_size, resolution):
    population = np.zeros((population_size, 16), dtype=np.int32)

    max_power = 226
    max_mass = 364

    res_x = resolution[0]
    res_y = resolution[1]
    res_z = resolution[2]

    #initial position
    population[:,0] = np.random.randint(res_x, size=population_size)[:]

    population[:,1] = np.random.randint(res_y, size=population_size)[:]

    population[:,2] = np.random.randint(res_z, size=population_size)[:]
    
    #velocity + position provides vector - i.e. initial debt
    population[:,3] = 0
    population[:,4] = 0
    population[:,5] = 0

    #mass, power randomized? should be part of genome - eventually?
    population[:,6] = np.random.randint(max_power/4, max_power, size=population_size)[:]

    population[:,7] = np.random.randint(max_mass/4, max_mass, size=population_size)[:]
    
    #genomic weights to be used as bytestrings
    population[:,8:16] = 0 

    return population

In [ ]:
def im(population, resolution):
    res_x = resolution[0]
    res_y = resolution[1]
    
    flat_world = np.zeros((res_x,res_y,3), dtype=np.uint8)
    
    flat_world[population[:,0],population[:,1],0] = 255
    flat_world[population[:,0],population[:,1],1] = 255
    flat_world[population[:,0],population[:,1],2] = 255
    return flat_world

In [ ]:
def draw(population, resolution, count):
    res_x = resolution[0]
    res_y = resolution[1]

    flat_world = np.zeros((res_x, res_y), dtype=bool)
    flat_world[population[:,0],population[:,1]] = 1
    scp.imsave("./out/image/frame_{0:05d}.png".format(count),flat_world.astype(bool))

In [ ]:
def form_world(population, resolution):
    res_x = resolution[0]
    res_y = resolution[1]
    res_z = resolution[2]

    world = np.zeros((res_x, res_y, res_z), dtype=np.int32)
    world[population[:,0],population[:,1], population[:,2]] = 1
    return world

In [ ]:
def save_as_hd5(population, resolution, filepath=None):
    if filepath:
      h5_out = tables.open_file(filepath, mode='w', title="Starlings")
    else:
      h5_out = tables.open_file('./starlings.h5', mode='w', title="Starlings")
    root = h5_out.root
    h5_out.create_array(root, "population", population)
    h5_out.create_array(root, "resolution", resolution)
    h5_out.close()

In [ ]:
def read_from_hd5(filepath=None):
    if filepath:
      h5_in = tables.open_file(filepath, mode='r')
    else:
      h5_in = tables.open_file('./starlings.h5', mode='r')
    population = h5_in.get_node("/population").read()
    resolution = h5_in.get_node("/resolution").read()
    h5_in.close()
    return population, resolution

In [ ]:
class OpenCl(object):
    def __init__(self):
        self.ctx, self.queue = self.cl_init()
        self.program = self.cl_load_program("./kernal.cl")

    def cl_init(self):
        platforms = cl.get_platforms()
        ctx = cl.create_some_context()
        queue = cl.CommandQueue(ctx)
        return ctx, queue

    def cl_load_program(self, filepath):
        f = open(filepath, 'r')
        fstr = "".join(f.readlines())
        program = cl.Program(self.ctx, fstr).build()
        return program

    def cl_load_data(self, population, world):
        mf = cl.mem_flags
        out = cl.Buffer(self.ctx, mf.WRITE_ONLY, population.nbytes)
        population_cl = cl.Buffer(self.ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=population)
        world_cl = cl.image_from_array(self.ctx, world, mode="r")
        return population_cl, world_cl, out

    def execute(self, num, population, world, inner_rad, outer_rad):
        population_cl, world_cl, out = self.cl_load_data(population, world)

        world_x = np.int32(world.shape[0])
        world_y = np.int32(world.shape[1])
        world_z = np.int32(world.shape[2])

        constants = np.asarray([0, 0], dtype=np.int32)

        ret = np.zeros_like(population)        

        global_size = ((num),)
        local_size = None

        kernalargs = (
                      population_cl,
                      out,
                      world_cl,
                      world_x, world_y, world_z,
                      inner_rad, outer_rad
                     )

        image_sequence = []

        self.program.knn(self.queue, global_size, local_size, *(kernalargs)).wait()
        cl.enqueue_copy(self.queue, ret, out)
        #cl.enqueue_read_buffer(self.queue, out, ret)


        #supersitious drivel!
        out.release()
        population_cl.release()
        world_cl.release()
        
        #self.queue.finish()
        return ret

In [ ]:
def plot_(starlings, resolution):
    plt.figure(figsize = (20,20))
    
    plt.imshow(im(starlings,resolution), interpolation='nearest')
    
    plt.plot([starlings[1066,1],(starlings[1066,1] - starlings[1066,4])], 
             [starlings[1066,0],(starlings[1066,0] - starlings[1066,3])], 
             lw="3", c='red')
    plt.plot([starlings[1067,1],(starlings[1067,1] - starlings[1067,4])], 
             [starlings[1067,0],(starlings[1067,0] - starlings[1067,3])], 
             lw="3", c='green')
    plt.plot([starlings[1068,1],(starlings[1068,1] - starlings[1068,4])], 
             [starlings[1068,0],(starlings[1068,0] - starlings[1068,3])]
             , lw="3", c='magenta')
    
    plt.scatter((starlings[1066,1] - starlings[1066,4]),
                (starlings[1066,0] - starlings[1066,3]), 
                s=600, c='red', alpha=.5, marker='d')
    plt.scatter((starlings[1067,1] - starlings[1067,4]),
                (starlings[1067,0] - starlings[1067,3]), 
                s=600, c='green', alpha=.5, marker='d')
    plt.scatter((starlings[1068,1] - starlings[1068,4]),
                (starlings[1068,0] - starlings[1068,3]), 
                s=600, c='magenta', alpha=.5, marker='d')
    
    plt.scatter(starlings[1066,1], starlings[1066,0], s=600, c='red', )
    plt.scatter(starlings[1067,1], starlings[1067,0], s=600, c='green')
    plt.scatter(starlings[1068,1], starlings[1068,0], s=600, c='magenta')
    
    return plt

In [ ]:
num = 640 * 480
resolution = [480, 640, 1]

starlings = init_data(num, resolution)
world = form_world(starlings, resolution)

opcl = OpenCl()

count = 0

inner_rad = np.int32(16)
outer_rad = np.int32(32)

stop = 24 * 60

frames = []

while count < stop:

      frames.append(starlings)

      draw(starlings, resolution, count)
      
      plot_(starlings, resolution).show()

      try:
        starlings = opcl.execute(num, starlings, world, inner_rad, outer_rad)
      except cl.RuntimeError, e:
        print str(e)
        np.set_printoptions(threshold=np.nan, linewidth=512)
        print starlings[:,0:15]
        if filepath and filepath[0]:
          save_as_hd5(starlings, resolution, filepath[0])
        else:
          save_as_hd5(starlings, resolution)
        raise
      world = form_world(starlings, resolution)
    
      count += 1
        
frames = np.asarray(frames)


[x,y,z,vel_x,vel_y,vel_z,mass,power,coheded,separated,desire_debug,accel_frame_debug]